# Comparing Train and Test Sets
An exploration on covariance shift in the Santander Value Prediction datasets

In [5]:
# Load libraries:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pickle as pkl

import time
import h5py

from catboost import CatBoostClassifier

from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

## Helper Functions:

In [2]:
# Load h5py file
def loadh5(fname, dname):
    h5f = h5py.File(fname, 'r')
    data = h5f[dname][:]
    h5f.close()
    return data

In [3]:
# Load pickle file
def loadpickle(fname):
    with open(fname, 'rb') as handle:
        data = pkl.load(handle)
    return data

In [4]:
# Function for loading data
def load_vanilla(debug=False):
    '''
    Function for loading data
    Input: debug=False if full dataset is desired
    '''
    data_path = '../../data/'
    train_dname = 'train_s0'
    test_dname = 'test_s0'
    f_ext = '_vanilla.h5'
    
    print 'Loading Stage 0 vanilla train and test datasets...'
    # Load h5py data
    train_data = loadh5(data_path + train_dname + f_ext, train_dname)
    test_data = loadh5(data_path + test_dname + f_ext, test_dname)
    # Load dataframe indexes
    train_idx = loadpickle(data_path + 'train_idx.pkl')
    test_idx = loadpickle(data_path + 'test_idx.pkl')
    # Load dataframe column names
    train_cols = loadpickle(data_path + 'train_cols.pkl')
    test_cols = loadpickle(data_path + 'test_cols.pkl')
    # Create dataframes
    train_df = pd.DataFrame(data=train_data, index=train_idx, columns=train_cols)
    test_df = pd.DataFrame(data=test_data, index=test_idx, columns=test_cols)
    
    if debug:
        return 
    
    return train_df, test_df

## Main Script:

In [ ]:
# Load data
load_start = time.time()


print 'Loading completed in %s seconds'%(time.time()-load_start)